In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.feature_selection import SelectKBest, f_regression, mutual_info_regression, RFE
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score, confusion_matrix
from sklearn.ensemble import RandomForestRegressor
from catboost import CatBoostRegressor, Pool, cv
import optuna
from sklearn.linear_model import LogisticRegression
from sklearn.multioutput import MultiOutputClassifier
from sklearn.pipeline import Pipeline
import mrmr
from mrmr import mrmr_regression
import warnings

# Set random seed for reproducibility
np.random.seed(42)

# Import and Prepare Data

In [2]:
# Load the data
file_path = "features-Master.csv"
data = pd.read_csv(file_path)

# Shuffle the data
shuffled = data.sample(frac=1, random_state=42).reset_index(drop=True)
data_shuffled = shuffled.iloc[:, 4:]
labels_shuffled = shuffled["Comfort Score"]

# Class to Accomodate MRMR Feature Selection Structure

In [3]:
class MRMRTransformer:
    def __init__(self, k_features):
        self.k_features = k_features
        self.selected_features = None
        self.column_names = None
    
    def fit(self, X, y):
        # Convert to DataFrame if not already
        if not isinstance(X, pd.DataFrame):
            X = pd.DataFrame(X)
        
        # Reset indices to avoid alignment issues
        X = X.reset_index(drop=True)
        y = pd.Series(y).reset_index(drop=True)
        
        self.column_names = X.columns.tolist()
        try:
            self.selected_features = mrmr_regression(X, y, K=self.k_features)
        except:
            # Fallback to random features if MRMR fails
            self.selected_features = np.random.choice(X.columns, size=min(self.k_features, len(X.columns)), replace=False)
        return self
    
    def transform(self, X):
        if not isinstance(X, pd.DataFrame):
            X = pd.DataFrame(X, columns=self.column_names)
        return X[self.selected_features]

# Train/Test Split

In [4]:
# For regression (using stratified split based on binned target)
X_train, X_test, y_train, y_test = train_test_split(
    data_shuffled, 
    labels_shuffled, 
    test_size=0.2, 
    stratify=labels_shuffled,
    random_state=42
)

# Optimize Feature Selection and Catboost Parameters

In [ ]:
X = X_train.copy()
y = y_train.copy().values

def objective(trial):
    # 1. Feature Selection
    fs_method = trial.suggest_categorical('feature_selection', ['RFE', 'MRMR', 'None'])
    if fs_method != 'None':
        k_features = trial.suggest_int('k_features', 5, X.shape[1], step = 10) # Can edit step size and log bool (log is strange though)
        if fs_method == 'RFE':
            rfe_step = trial.suggest_float('rfe_step', 0.1, 1.0, step=0.1) #### Can edit step size
            estimator = RandomForestRegressor(
                n_estimators=trial.suggest_int('rfe_n_estimators', 50, 200),
                max_depth=trial.suggest_int('rfe_max_depth', 3, 10),
                random_state=42
            )
            selector = RFE(estimator, n_features_to_select=k_features, step=rfe_step)
        elif fs_method == 'MRMR':
            selector = MRMRTransformer(k_features=k_features)
    else:
        selector = 'passthrough'

    # 2. CatBoost Parameters
    params = {
        'iterations': trial.suggest_int('iterations', 100, 1000),
        'depth': trial.suggest_int('depth', 3, 10),
        'learning_rate': trial.suggest_float('learning_rate', 1e-3, 0.3, log=True),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1e-8, 10.0, log=True),
        'random_state': 42,
        'verbose': False
    }
    model = CatBoostRegressor(**params)

    # 3. Pipeline 
    pipeline = Pipeline([
        ('feature_selection', selector),
        ('model', model)
    ])

    #4. Cross-validation
    cv = KFold(n_splits=5, shuffle=True, random_state=42)
    try:
        scores = cross_val_score(pipeline, X, y, cv=cv, scoring='r2', n_jobs=1)
        return np.mean(scores)
    except Exception:
        return -np.inf

# Run Optuna Study
study = optuna.create_study(direction='maximize') # sampler = optuna.samplers.QMCSampler())

study.optimize(objective, n_trials=5, show_progress_bar=True, n_jobs=6)

# Best result
print("Best trial:")
trial = study.best_trial
print(f"R²: {trial.value:.4f}")
print("Params: ")
for key, value in trial.params.items():
    print(f"  {key}: {value}")

[I 2025-08-27 10:08:16,525] A new study created in memory with name: no-name-28de511e-9905-4a6a-be96-5eddca5c46ac


  0%|          | 0/5 [00:00<?, ?it/s]

100%|██████████| 165/165 [00:27<00:00,  6.00it/s]


[I 2025-08-27 10:10:15,944] Trial 1 finished with value: 0.17443952264052712 and parameters: {'feature_selection': 'RFE', 'k_features': 75, 'rfe_step': 0.8, 'rfe_n_estimators': 177, 'rfe_max_depth': 4, 'iterations': 748, 'depth': 3, 'learning_rate': 0.030225419478809152, 'l2_leaf_reg': 0.003761494770139977}. Best is trial 1 with value: 0.17443952264052712.


[I 2025-08-27 10:10:52,444] Trial 3 finished with value: 0.21025572404751447 and parameters: {'feature_selection': 'None', 'iterations': 203, 'depth': 7, 'learning_rate': 0.030761992256186997, 'l2_leaf_reg': 4.563000684259326e-08}. Best is trial 3 with value: 0.21025572404751447.


100%|██████████| 165/165 [00:21<00:00,  7.62it/s]


[I 2025-08-27 10:12:24,564] Trial 0 finished with value: 0.19582284522618304 and parameters: {'feature_selection': 'MRMR', 'k_features': 165, 'iterations': 322, 'depth': 7, 'learning_rate': 0.06799633531731222, 'l2_leaf_reg': 1.6659609751037856e-08}. Best is trial 3 with value: 0.21025572404751447.
[I 2025-08-27 10:15:05,527] Trial 4 finished with value: 0.1917746500010245 and parameters: {'feature_selection': 'RFE', 'k_features': 205, 'rfe_step': 0.2, 'rfe_n_estimators': 185, 'rfe_max_depth': 8, 'iterations': 817, 'depth': 7, 'learning_rate': 0.002726226840508819, 'l2_leaf_reg': 0.0004428015715506437}. Best is trial 3 with value: 0.21025572404751447.
[I 2025-08-27 10:15:50,362] Trial 2 finished with value: 0.16073564077264435 and parameters: {'feature_selection': 'RFE', 'k_features': 295, 'rfe_step': 0.9, 'rfe_n_estimators': 200, 'rfe_max_depth': 7, 'iterations': 882, 'depth': 7, 'learning_rate': 0.14548579742576695, 'l2_leaf_reg': 1.833210817248078e-08}. Best is trial 3 with value: 0

In [8]:
import optuna.visualization as vis

# 1. Parallel Coordinate Plot (Best for seeing parameter relationships)
fig = vis.plot_parallel_coordinate(
    study,
    params=[
        'feature_selection',
        'k_features',
        'depth',
        'learning_rate'
    ],
    target_name="R² Score"
)
fig.show()

# 2. Parameter Importance Plot
fig = vis.plot_param_importances(study, target_name="R² Score")
fig.show()

# 3. Slice Plot (Best for seeing individual parameter effects)
fig = vis.plot_slice(
    study,
    params=[
        'depth',
        'learning_rate',
        'k_features'
    ],
    target_name="R² Score"
)
fig.show()

# Apply Feature Selection to Training & Test Data

In [ ]:
# apply your feature selection code from before
best_fs_method = study.best_params.get('feature_selection', 'None')

if best_fs_method != 'None':
    k_features = study.best_params['k_features']
    if best_fs_method == 'RFE':
        estimator = RandomForestRegressor(
            n_estimators=study.best_params.get('rfe_n_estimators', 100),
            max_depth=study.best_params.get('rfe_max_depth', 5),
            random_state=42
        )
        selector = RFE(
            estimator, 
            n_features_to_select=k_features,
            step=study.best_params.get('rfe_step', 1)
        )
    elif best_fs_method == 'MRMR':
        selector = MRMRTransformer(k_features=k_features)
    
    selector.fit(X, y)
    if hasattr(selector, 'get_support'):  # For SelectKBest/RFE
        selected_features = X.columns[selector.get_support()]
    else:  # For MRMRTransformer
        selected_features = selector.selected_features
    X_best = X[selected_features]
else:
    X_best = X
    selected_features = X.columns

# Apply the same feature selection to test data
if best_fs_method != 'None':
    if best_fs_method == 'MRMR':
        X_test_final = X_test[selected_features]
    else:
        X_test_final = selector.transform(X_test)  # Use the already fitted selector
        if isinstance(X_test, pd.DataFrame):
            X_test_final = pd.DataFrame(X_test_final, columns=selected_features)
else:
    X_test_final = X_test

# Create Catboost Model with Optimized Parameters and Fit Model

In [9]:
best_model = CatBoostRegressor(
    iterations=study.best_params['iterations'],
    depth=study.best_params['depth'],
    learning_rate=study.best_params['learning_rate'],
    l2_leaf_reg=study.best_params['l2_leaf_reg'],
    random_state=42,
    verbose=False
)

# Train on full imputed data
best_model.fit(X_best, y)

# Run Model and Get Outcomes

In [10]:
# Make predictions
y_pred = best_model.predict(X_test_final)

# Calculate metrics
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)

print("\nFinal Model Evaluation on Test Set:")
print(f"RMSE: {rmse:.4f}")
print(f"R²: {r2:.4f}")

# Calculate accuracy within ±1 point
correct = np.sum(np.abs(y_test - y_pred) <= 1)
accuracy = correct / len(y_test)
print(f"Accuracy within ±1 point: {accuracy:.4f}")


Final Model Evaluation on Test Set:
RMSE: 1.1496
R²: 0.1765
Accuracy within ±1 point: 0.6250
